In [1]:
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
import numpy as np

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [3]:
# 处理数据
x_train = x_train.reshape([-1,28,28,1]).astype(np.float32)
x_test = x_test.reshape([-1,28,28,1]).astype(np.float32)

y_train = y_train.reshape([-1,1])
y_test = y_test.reshape([-1,1])

one_hot = OneHotEncoder(categories='auto')
y_train = one_hot.fit_transform(y_train).toarray()
y_test = one_hot.transform(y_test).toarray()

In [4]:
INPUT_WIDTH=28
INPUT_HEIGHT=28
INPUT_CHANNEL=1
OUTPUT_NODE=10
BATCH_SIZE=50

1. 定义需要的函数

In [5]:
def weight_variable(shape):
    '''
    权重
    '''
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    '''
    偏置
    '''
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


def conv2d(x, W):
    '''
    2d卷积层
    '''
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')


def max_pool_2x2(x):
    '''
    池化层
    '''
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

def get_batch(data, step, batch_size):
    '''
    辅助函数，用于获得批量数据
    '''
    result = []
    for elem in data:
        size = len(elem)
        start = (step*batch_size)%size
        end = (step*batch_size+batch_size)%size
        result.append(elem[start:end])
    return result

2. 定义网络

In [6]:
# 定义输入输出
x = tf.placeholder(tf.float32, shape=[None, INPUT_WIDTH, INPUT_HEIGHT, INPUT_CHANNEL])
y_ = tf.placeholder(tf.float32, shape=[None, OUTPUT_NODE])

# 定义第一层网络
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

# 定义第二层卷积网络
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

# 两层卷积层之后是全连接层
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# 使用dropout，keep_prob是一个占位符，训练时为0.5，测试时为1.0
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# 吧1024维转成10维
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

# 定义交叉熵损失函数
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_conv, labels=y_))
# 定义优化器
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# 定义测试准确率
corrent_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(corrent_prediction, tf.float32))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [7]:
sess = tf.InteractiveSession()

In [8]:
sess.run(tf.global_variables_initializer())

In [ ]:
for step in range(20000):
    batch_train_x, batch_train_y = get_batch([x_train, y_train], step, BATCH_SIZE)
    train_feed={x:batch_train_x, y_:batch_train_y, keep_prob:0.5}
    test_feed={x:x_test, y_:y_test, keep_prob:1.0}
    # 每100步报告一次在测试集上的准确率
    if step % 100 == 0:
        train_accuracy = accuracy.eval(feed_dict=train_feed)
        test_accuracy = accuracy.eval(feed_dict=test_feed)
        print('after {} step, train accuracy is {}, test accuracy is {}'.format(step, train_accuracy, test_accuracy))
#         print('after {} step, train accuracy is {}, test accuracy is '.format(step, train_accuracy))
    sess.run(train_step, feed_dict=train_feed)
    
# 训练结束时进行报告
train_accuracy = accuracy.eval(feed_dict=train_feed)
test_accuracy = accuracy.eval(feed_dict=test_feed)
print('after {} step, train accuracy is {}, test accuracy is {}'.format(step, train_accuracy, test_accuracy))